<a href="https://colab.research.google.com/github/LinkRoxo/TCC/blob/main/Scripts/OCR_nos_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**INIT**

In [ ]:
!pip install opencv-python==4.8.0.76
!sudo apt install tesseract-ocr
!pip install pytesseract==0.3.9
!apt-get install tesseract-ocr-por

In [2]:
import pytesseract
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from PIL import ImageFont, ImageDraw, Image

fonte = '/content/drive/MyDrive/Colab Notebooks/Fonte/Calibri/Calibri.ttf'

In [3]:
import pandas as pd

In [4]:
def caixa_texto(resultado, img, cor = (255, 100, 0)):
  x = resultado['left'][i]
  y = resultado['top'][i]
  w = resultado['width'][i]
  h = resultado['height'][i]

  cv2.rectangle(img, (x, y), (x+w, y+h), cor, 2)

  return x, y, img

In [5]:
def escreve_texto(texto, x, y, img, fonte, tamanho_texto=20):
  fonte = ImageFont.truetype(fonte, tamanho_texto)
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text((x, y - tamanho_texto), texto, font = fonte, fill=(0,0,255,255))
  img = np.array(img_pil)
  return img

###CODIGO TESSERACT


testes de bibliotecas e ambiente

In [6]:
# @title Linguas
!tesseract --list-langs

List of available languages (3):
eng
osd
por


In [7]:
cv2.__version__

'4.8.0'

In [ ]:
!tesseract --help-psm

CONFIG TESSERACT

In [16]:
gibby = cv2.imread('/content/Fatura gibby-1.png')
pedaço_gibby = cv2.imread('/content/pedaço_gibby.png')
pedaço_gibby2 = cv2.imread('/content/pedaço_gibby2.png')

In [ ]:
text = pytesseract.image_to_string(gibby, lang='por', config='--psm 3')
text_data = pytesseract.image_to_data(gibby, config='--psm 3', lang='por', output_type=pytesseract.Output.DICT)
#text2 = pytesseract.image_to_string(pedro, lang='por')
#text3 = pytesseract.image_to_string(john, lang='por')
text_data

In [ ]:
dframe = pd.DataFrame(text_data)

limite_confianca = 80

palavras_por_bloco = {}

for i, (texto, num_bloco, conf) in enumerate(zip(text_data['text'], text_data['block_num'], text_data['conf'])):
    if i == 0:
        continue  # Ignorar o cabeçalho

    if int(conf) >= limite_confianca:
        if num_bloco not in palavras_por_bloco:
            palavras_por_bloco[num_bloco] = []

        palavras_por_bloco[num_bloco].append(texto)

faca = False
# Agora você tem as palavras agrupadas por bloco
if faca == True:
  for num_bloco, palavras in palavras_por_bloco.items():
      print(f"Bloco {num_bloco}:")
      for palavra in palavras:
          print("                            "+palavra)

print(palavras_por_bloco[8])

In [ ]:
text2 = pytesseract.image_to_data(pedaço_gibby, config='--psm 3', lang='por', output_type=pytesseract.Output.DICT)
text2

### **IMAGEM PRINTADA** 🎉

---



In [ ]:
gibby_resultado = gibby
for i in range(len(text_data['text'])):
  texto = text_data['text'][i]
  if int(text_data['conf'][i]) > 70:
    x, y, img = caixa_texto(text_data, gibby_resultado)
    #cv2.putText(gibby_resultado, texto, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255))
    gibby_resultado = escreve_texto(texto, x, y, gibby_resultado, fonte)
cv2_imshow(gibby_resultado)


###**spaCy**

In [ ]:
!pip install -U pip setuptools wheel
#!pip install -U 'spacy[transformers,lookups]'
!pip install -U 'spacy[cuda-autodetect]'
!python -m spacy download pt_core_news_lg

In [22]:
import spacy

nlp = spacy.load('pt_core_news_lg')
nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']

In [23]:
def laço(texto):
  spacy.displacy.render(texto, style='ent', jupyter=True)

In [24]:
def unificador(bloco):
  delimitador = " "  # Espaço em branco entre as palavras

  string_unificada = delimitador.join(bloco)
  return string_unificada

In [ ]:
for i in palavras_por_bloco:
  text = laço(nlp(unificador(palavras_por_bloco[i])))